In [1]:
import pandas as pd
import numpy as np
from utils import *

In [2]:
df = read_dd_data('../data/PD_data')
# why is name the same

In [3]:
df

,rcert,runcert,event_prob,choice,condition,rt,odds,participant
0,5.0,9.50,0.10,0.0,1.0,2013.5,9.000000,bh348lli7
1,10.0,11.11,0.90,0.0,1.0,514.4,0.111111,bh348lli7
2,1.0,1.10,0.50,0.0,1.0,424.2,1.000000,bh348lli7
3,1.0,31.00,0.25,0.0,1.0,398.2,3.000000,bh348lli7
4,20.0,80.00,0.50,0.0,1.0,538.3,1.000000,bh348lli7
...,...,...,...,...,...,...,...,...
9795,-10.0,-100.00,0.10,0.0,2.0,1504.1,9.000000,u3yyfob1i
9796,-10.0,-40.00,0.25,1.0,2.0,1856.4,3.000000,u3yyfob1i
9797,-5.0,-9.50,0.10,1.0,2.0,1911.6,9.000000,u3yyfob1i
9798,-50.0,-100.00,0.75,0.0,2.0,2848.4,0.333333,u3yyfob1i


In [ ]:
'''
example of hierachical modeling with pyMC just for choices with prospect theory
'''
with pm.Model() as model:
    # Hierarchical priors for probability weighting parameter (γ)
    mu_gamma = pm.Normal("mu_gamma", mu=0, sigma=1)  # Group-level mean log(γ)
    sigma_gamma = pm.HalfNormal("sigma_gamma", sigma=1)  # Group-level variability
    
    # Individual γ values (log-normal to ensure positivity)
    participant_ids = df['participant'].astype("category").cat.codes.values
    num_participants = len(np.unique(participant_ids))
    
    gamma_log = pm.Normal("gamma_log", mu=mu_gamma, sigma=sigma_gamma, shape=num_participants)
    gamma = pm.Deterministic("gamma", pm.math.exp(gamma_log))  # Convert to positive space
    
    # Softmax noise parameter
    beta = pm.HalfNormal("beta", sigma=5)  # Controls randomness of choices

    # Compute subjective probability weighting
    p = df['event_prob'].values
    subjective_p = (p**gamma[participant_ids]) / ((p**gamma[participant_ids] + (1 - p)**gamma[participant_ids])**(1/gamma[participant_ids]))
    
    # Expected values
    V_cert = df['rcert'].values
    V_uncert = df['runcert'].values * subjective_p
    
    # Logistic choice model
    p_uncertain = pm.Deterministic("p_uncertain", 1 / (1 + pm.math.exp(-beta * (V_uncert - V_cert))))
    
    # Likelihood (observed choices)
    choice_obs = pm.Bernoulli("choice", p=p_uncertain, observed=(df['choice'].values))

    # Sampling
    trace = pm.sample(2000, tune=1000, target_accept=0.9, return_inferencedata=True)